<a href="https://colab.research.google.com/github/staracode/portfolio/blob/main/Pan_cancer_Nuceli_seg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


Processed data stored here https://huggingface.co/datasets/longevity-db/pan-cancer-nuclei-seg

Data
https://faspex.cancerimagingarchive.net/aspera/faspex/public/package?context=eyJyZXNvdXJjZSI6InBhY2thZ2VzIiwidHlwZSI6ImV4dGVybmFsX2Rvd25sb2FkX3BhY2thZ2UiLCJpZCI6Ijc2NiIsInBhc3Njb2RlIjoiMDgwNjQ5ZDRmOWRjZjMwMzllMDMyN2Y5Njk2MTU5NTkxNWY4MjNmMiIsInBhY2thZ2VfaWQiOiI3NjYiLCJlbWFpbCI6ImhlbHBAY2FuY2VyaW1hZ2luZ2FyY2hpdmUubmV0In0=&redirected=true&authenticated=true

In [ ]:
!cp /content/drive/MyDrive/TCGA-2F-A9KO-01Z-00-DX1.195576CF-B739-4BD9-B15B-4A70AE287D3E.svs.tar.gz /content/TCGA-2F-A9KO.svs.tar.gz
!tar xf /content/TCGA-2F-A9KO.svs.tar.gz

In [ ]:
import duckdb

In [ ]:
df = duckdb.sql("""
WITH T AS (
SELECT
    CAST(AreaInPixels AS UINT16) as AreaInPixels,
    CAST(PhysicalSize AS UINT16) as PhysicalSize,
    (Polygon[2:-2]).split(':').list_transform(x -> CAST(x AS FLOAT)) as temp
FROM read_csv_auto('Downloads/skcm_polygonTCGA-Z2-AA3V-01Z-00-DX1.776C7396-2B67-472B-A332-2E2CCD28B110.svs/100001*.csv', union_by_name=True)
) SELECT
    AreaInPixels,
    PhysicalSize,
    CAST(list_zip(temp[1:-:2], temp[2:-:2]) AS struct(x float, y float)[]) as Polygon
FROM T
""").df()

In [ ]:
df = duckdb.sql("""
WITH T AS (
SELECT
    CAST(AreaInPixels AS UINT16) as AreaInPixels,
    CAST(PhysicalSize AS UINT16) as PhysicalSize,
    string_split(regexp_replace(Polygon, '[\\[\\]]', ''), ':') as coords_array
FROM read_csv_auto('/content/blca_polygon/TCGA-2F-A9KO-01Z-00-DX1.195576CF-B739-4BD9-B15B-4A70AE287D3E.svs/100001*.csv', union_by_name=True)
),
Parsed AS (
SELECT
    AreaInPixels,
    PhysicalSize,
    list_transform(
        range(0, len(coords_array), 2),
        i -> {'x': CAST(coords_array[i] AS FLOAT), 'y': CAST(coords_array[i+1] AS FLOAT)}
    ) as Polygon
FROM T
WHERE len(coords_array) >= 6
)
SELECT * FROM Parsed
""").df()

In [ ]:
df["Polygon"]

,Polygon
0,"[{'x': None, 'y': 101556.0}, {'x': 102289.0, '..."
1,"[{'x': None, 'y': 101196.0}, {'x': 102667.0, '..."
2,"[{'x': None, 'y': 102530.0}, {'x': 103316.0, '..."
3,"[{'x': None, 'y': 100370.0}, {'x': 103673.0, '..."
4,"[{'x': None, 'y': 102094.0}, {'x': 103696.0, '..."
...,...
84379,"[{'x': None, 'y': 102976.0}, {'x': 99028.0, 'y..."
84380,"[{'x': None, 'y': 103013.0}, {'x': 99045.0, 'y..."
84381,"[{'x': None, 'y': 100983.0}, {'x': 99203.0, 'y..."
84382,"[{'x': None, 'y': 100968.0}, {'x': 99244.0, 'y..."


In [ ]:
from shapely.geometry import Polygon
from scipy.spatial import ConvexHull
from skimage.measure import regionprops
import numpy as np

In [ ]:
import numpy as np
from shapely.geometry import Polygon

def calculate_aspect_ratio(points):
    """
    Fast bounding box aspect ratio calculation.
    Optimized for large-scale analysis (11K+ slides).

    Args:
        points: List of (x, y) coordinate tuples

    Returns:
        float: Aspect ratio (>= 1.0)
    """
    # Convert to numpy array for vectorized operations
    points_array = np.array(points)

    # Get min/max coordinates in one operation
    x_coords = points_array[:, 0]
    y_coords = points_array[:, 1]

    width = np.max(x_coords) - np.min(x_coords)
    height = np.max(y_coords) - np.min(y_coords)

    # Avoid division by zero
    if min(width, height) == 0:
        return 1.0

    # Return aspect ratio (always >= 1.0)
    return max(width, height) / min(width, height)

def extract_aging_features(polygon_coords):
    """
    Your optimized feature extraction function.
    Fast and reliable for massive datasets.
    """
    # Parse coordinates
    polygon_coords_v2 = [(d["x"], d["y"]) for d in polygon_coords]
    points = polygon_coords_v2
    poly = Polygon(points)

    # Tier 1 features (most important)
    features = {
        'area': poly.area,
        'perimeter': poly.length,
        'convexity': poly.area / poly.convex_hull.area,
        'aspect_ratio': calculate_aspect_ratio(points),  # Fast bounding box method
        'circularity': 4 * np.pi * poly.area / (poly.length ** 2)
    }

    return features

In [ ]:
features1 = df["Polygon"].map(extract_aging_features)

In [ ]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
import warnings
warnings.filterwarnings('ignore')

def robust_extract_aging_features(polygon_coords):
    """
    Handle numpy array format polygon data with None values
    """
    try:
        # Convert numpy array to list if needed
        if isinstance(polygon_coords, np.ndarray):
            polygon_coords = polygon_coords.tolist()

        # Handle None or empty polygons
        if not polygon_coords or len(polygon_coords) < 3:
            return {
                'area': 0, 'perimeter': 0, 'convexity': 0,
                'aspect_ratio': 1, 'circularity': 0,
                'error': f'insufficient_points_{len(polygon_coords) if polygon_coords else 0}'
            }

        # Clean coordinates - remove None values and invalid entries
        clean_coords = []
        for point in polygon_coords:
            try:
                # Handle different formats
                if isinstance(point, dict):
                    x, y = point.get('x'), point.get('y')
                elif isinstance(point, (list, tuple)) and len(point) >= 2:
                    x, y = point[0], point[1]
                else:
                    continue  # Skip invalid point formats

                # Check if coordinates are valid
                if x is not None and y is not None:
                    x_float, y_float = float(x), float(y)
                    # Check for finite values (not inf, not nan)
                    if np.isfinite(x_float) and np.isfinite(y_float):
                        clean_coords.append((x_float, y_float))

            except (ValueError, TypeError, AttributeError):
                continue  # Skip problematic coordinates

        # Check if we have enough valid coordinates
        if len(clean_coords) < 3:
            return {
                'area': 0, 'perimeter': 0, 'convexity': 0,
                'aspect_ratio': 1, 'circularity': 0,
                'error': f'too_few_valid_coords_{len(clean_coords)}'
            }

        # Remove consecutive duplicate points
        unique_coords = []
        for coord in clean_coords:
            if not unique_coords or coord != unique_coords[-1]:
                unique_coords.append(coord)

        if len(unique_coords) < 3:
            return {
                'area': 0, 'perimeter': 0, 'convexity': 0,
                'aspect_ratio': 1, 'circularity': 0,
                'error': f'duplicate_coords_{len(unique_coords)}'
            }

        # Create polygon safely
        try:
            poly = Polygon(unique_coords)

            # Check if polygon is valid
            if not poly.is_valid:
                # Try to fix simple self-intersection issues
                poly = poly.buffer(0)
                if not poly.is_valid:
                    return {
                        'area': 0, 'perimeter': 0, 'convexity': 0,
                        'aspect_ratio': 1, 'circularity': 0,
                        'error': 'invalid_geometry'
                    }
        except Exception as geom_error:
            return {
                'area': 0, 'perimeter': 0, 'convexity': 0,
                'aspect_ratio': 1, 'circularity': 0,
                'error': f'geometry_creation_failed'
            }

        # Calculate features safely
        try:
            area = poly.area
            perimeter = poly.length

            # Check for zero area/perimeter
            if area <= 0 or perimeter <= 0:
                return {
                    'area': float(area), 'perimeter': float(perimeter), 'convexity': 0,
                    'aspect_ratio': 1, 'circularity': 0,
                    'error': 'zero_area_or_perimeter'
                }

            # Convexity (safe calculation)
            try:
                convex_hull_area = poly.convex_hull.area
                convexity = area / convex_hull_area if convex_hull_area > 0 else 0
            except:
                convexity = 0

            # Aspect ratio using bounding box (fastest and most reliable)
            try:
                coords_array = np.array(unique_coords)
                x_coords = coords_array[:, 0]
                y_coords = coords_array[:, 1]

                width = np.max(x_coords) - np.min(x_coords)
                height = np.max(y_coords) - np.min(y_coords)

                # Avoid division by zero
                aspect_ratio = max(width, height) / max(min(width, height), 1e-10)
            except:
                aspect_ratio = 1.0

            # Circularity
            try:
                circularity = 4 * np.pi * area / (perimeter ** 2)
            except:
                circularity = 0

            return {
                'area': float(area),
                'perimeter': float(perimeter),
                'convexity': float(convexity),
                'aspect_ratio': float(aspect_ratio),
                'circularity': float(circularity),
                'error': None,
                'num_valid_coords': len(unique_coords)
            }

        except Exception as calc_error:
            return {
                'area': 0, 'perimeter': 0, 'convexity': 0,
                'aspect_ratio': 1, 'circularity': 0,
                'error': f'calculation_error'
            }

    except Exception as general_error:
        return {
            'area': 0, 'perimeter': 0, 'convexity': 0,
            'aspect_ratio': 1, 'circularity': 0,
            'error': f'general_error'
        }

# 🚀 COMPLETE PROCESSING PIPELINE
print("🔄 Starting robust polygon processing...")

# Step 1: Check your data format
print(f"Total polygons to process: {len(df):,}")
print(f"Sample polygon type: {type(df['Polygon'].iloc[0])}")

# Count None values in first coordinate to understand the scope
first_coords = [poly[0] if len(poly) > 0 else None for poly in df['Polygon'].head(100)]
none_count = sum(1 for coord in first_coords if coord is None or (isinstance(coord, dict) and coord.get('x') is None))
print(f"Estimated None coordinates in sample: {none_count}/100")

# Step 2: Process all polygons with progress tracking
print("\n🔄 Processing polygons...")
batch_size = 5000
total_batches = (len(df) + batch_size - 1) // batch_size

all_features = []
for batch_idx in range(total_batches):
    start_idx = batch_idx * batch_size
    end_idx = min((batch_idx + 1) * batch_size, len(df))

    batch_df = df.iloc[start_idx:end_idx]
    batch_features = batch_df['Polygon'].apply(robust_extract_aging_features)
    all_features.extend(batch_features.tolist())

    # Progress update every 10 batches
    if batch_idx % 10 == 0 or batch_idx == total_batches - 1:
        progress = (batch_idx + 1) / total_batches * 100
        print(f"  Progress: {progress:.1f}% ({end_idx:,}/{len(df):,} nuclei)")

# Step 3: Create results DataFrame
print("\n📊 Creating results DataFrame...")
features_df = pd.DataFrame(all_features)

# Step 4: Analyze results
print("\n✅ RESULTS SUMMARY")
print("="*50)

total_nuclei = len(features_df)
successful = features_df['error'].isna().sum()
failed = total_nuclei - successful

print(f"Total nuclei processed: {total_nuclei:,}")
print(f"Successful extractions: {successful:,} ({successful/total_nuclei*100:.1f}%)")
print(f"Failed extractions: {failed:,} ({failed/total_nuclei*100:.1f}%)")

# Show error breakdown
if failed > 0:
    print(f"\n❌ Error breakdown:")
    error_counts = features_df['error'].value_counts()
    for error, count in error_counts.head(10).items():
        if error is not None:
            print(f"  {error}: {count:,}")

# Statistics for successful extractions
valid_features = features_df[features_df['error'].isna()]
if len(valid_features) > 0:
    print(f"\n📈 Statistics for successful extractions:")
    print(f"  Mean area: {valid_features['area'].mean():.1f} pixels²")
    print(f"  Mean aspect ratio: {valid_features['aspect_ratio'].mean():.3f}")
    print(f"  Mean circularity: {valid_features['circularity'].mean():.3f}")
    print(f"  Mean convexity: {valid_features['convexity'].mean():.3f}")

    print(f"\n  Area range: {valid_features['area'].min():.1f} - {valid_features['area'].max():.1f}")
    print(f"  Aspect ratio range: {valid_features['aspect_ratio'].min():.3f} - {valid_features['aspect_ratio'].max():.3f}")

print(f"\n🎉 Processing complete! Features extracted for {successful:,} nuclei.")

# Step 5: Save results for your hackathon
print(f"\n💾 Your results are in 'features_df' DataFrame")
print(f"   Shape: {features_df.shape}")
print(f"   Columns: {list(features_df.columns)}")

# Quick preview
if len(valid_features) > 0:
    print(f"\n👀 Sample of successful results:")
    display_cols = ['area', 'aspect_ratio', 'circularity', 'convexity']
    print(valid_features[display_cols].head())

🔄 Starting robust polygon processing...
Total polygons to process: 84,384
Sample polygon type: <class 'numpy.ndarray'>
Estimated None coordinates in sample: 100/100

🔄 Processing polygons...
  Progress: 5.9% (5,000/84,384 nuclei)
  Progress: 64.7% (55,000/84,384 nuclei)
  Progress: 100.0% (84,384/84,384 nuclei)

📊 Creating results DataFrame...

✅ RESULTS SUMMARY
Total nuclei processed: 84,384
Successful extractions: 84,381 (100.0%)
Failed extractions: 3 (0.0%)

❌ Error breakdown:
  too_few_valid_coords_2: 1
  zero_area_or_perimeter: 1
  duplicate_coords_2: 1

📈 Statistics for successful extractions:
  Mean area: 425.7 pixels²
  Mean aspect ratio: 1.470
  Mean circularity: 0.625
  Mean convexity: 0.887

  Area range: 0.2 - 7995.5
  Aspect ratio range: 1.000 - 19.474

🎉 Processing complete! Features extracted for 84,381 nuclei.

💾 Your results are in 'features_df' DataFrame
   Shape: (84384, 7)
   Columns: ['area', 'perimeter', 'convexity', 'aspect_ratio', 'circularity', 'error', 'num_va